In [1]:
# Bibliotecas públicas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import os
import pyodbc
import csv
import functools
from typing import List

# Extração

In [24]:
query = f"""

    SELECT top 100     
    FROM [DBCAT].[dbo].[TBCAT_eSocial]
    
    """
    
connection = pyodbc.connect('Driver={SQL Server};Server=Marfim;Database=DBCAT;')
df = pd.read_sql_query(query, connection)

c:\Users\joao.reis\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Tratamento

In [25]:
# Converte tipos
df.vrsalfx = df.vrsalfx.astype('float')
df['durtrat'] = df['durtrat'].astype(int)

date_cols = ['dtadm', 'dtnascto', 'dtacid', 'dtobito', 'dtatendimento']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Cria colunas novas
df['idade_DTAcidente'] = ((df.dtacid - df.dtnascto) / np.timedelta64(1, 'Y')).round(0)
df['DTEmissaoCAT'] = pd.to_datetime(df.meta_row_key.str[:8], errors='coerce')
df['CDEmitenteCAT'] = '1'
df['NRCAT'] = df['meta_nr_recibo']

# Remove espaços em branco de colunas de texto
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Formata strings de horas
cols_horas = ['hracid', 'hrstrabantesacid', 'hratendimento']
for col in cols_horas:
    df[col] = df[col].apply(lambda x: x[:2] + ':' + x[2:] if not pd.isna(x) else np.NAN)

# Formata strings de datas
for col in date_cols:
    df[col] = df[col].dt.strftime('%d/%m/%Y')

# Identificação de registro raiz em caso de reaberturas / comunicações de óbito

In [26]:
# df['recibo_anterior'] = df.nrrecibo.combine_first(df.nrRecCatOrig)
dict_recibo_anterior = pd.Series(df.nrRecCatOrig.values, index=df.meta_nr_recibo).to_dict()

@functools.cache
def recibo_raiz(recibo):
    if pd.isna(dict_recibo_anterior[recibo]):
        return recibo
    else:
        try:
            return recibo_raiz(dict_recibo_anterior[recibo])
        except KeyError:
            return recibo

df['recibo_raiz'] = df.meta_nr_recibo.map(recibo_raiz)

In [27]:
# Mantém última CAT enviada, em caso de CATs de reabertura e comunicação de óbito
df = (df
      .sort_values('meta_nr_recibo', ascending=False)
      .groupby('recibo_raiz', dropna=False)
      .head(1)
      .reset_index()
      )

# Embelezamento da tabela (insere descrições, renomeia colunas)

In [55]:
cols_to_map  = ['tpacid',
                'tplocal_acidente',
                'tplograd_local_acidente',
                'municipio_local_acidente',
                'pais_local_acidente',
                'codagntcausador',
                'codsitgeradora',
                'codparteating',
                'lateralidade',
                'dsclesao',
                'codcid',
                'ideoc',
                'codcbo',
                'grauinstr',
                'racacor',
                'codcateg',
                'codcateg',
                'tpinsc',
                'localtabgeral_tpinsc',
                'cnae_localtabgeral',
                'municipio_empregador',
                'tpinsc_estab_local_acidente',
                'cnae_local_acidente',
                'municipio_estab_local_acidente',
                'CDEmitenteCAT',
                'iniciatcat',
                'tpcat',
                'indretif',
                'procemi',
                'inporte']

cols_integer_as_index = ['tpinsc', 'grauinstr', 'racacor', 'tpacid', 
                         'tpcat', 'lateralidade', 'iniciatcat', 'indretif',]


for new_col, map_dict in dict_new_columns.items():
    col_to_map = map_dict['col']
    dict_map = map_dict['map']
    df[new_col] = df[col_to_map].map(dict_map)

{1: 'CNPJ', 2: 'CPF', 3: 'CAEPF', 4: 'CNO'}

In [71]:
pd.read_csv('../data/input/aux_tables/CDEmitenteCAT.csv', dtype='object')

,CDEmitenteCAT,ds_CDEmitenteCAT
0,1,Empregador
1,2,Sindicato
2,3,Médico
3,4,Segurado/Dependente
4,5,Autoridade pública


In [3]:
# Tabelas de referência
tb_tpinsc = {1: 'CNPJ', 
             2: 'CPF', 
             3: 'CAEPF', 
             4: 'CNO',}

tb_grauinstr = {1:'Analfabeto, inclusive o que, embora tenha recebido instrução, não se alfabetizou',
                2:'Até o 5º ano incompleto do ensino fundamental (antiga 4ª série) ou que se tenha alfabetizado sem ter frequentado escola regular',
                3:'5º ano completo do ensino fundamental',
                4:'Do 6º ao 9º ano do ensino fundamental incompleto (antiga 5ª a 8ª série)',
                5:'Ensino fundamental completo',
                6:'Ensino médio incompleto',
                7:'Ensino médio completo',
                8:'Educação superior incompleta',
                9:'Educação superior completa',
                10: 'Pós-graduação completa',
                11: 'Mestrado completo',
                12: 'Doutorado completo'}

tb_racacor = {1: 'Branca',
              2: 'Preta',
              3: 'Parda',
              4: 'Amarela',
              5: 'Indígena',
              6: 'Não informado'}

tb_tpacid = {1: 'Típico',
             2: 'Doença',
             3: 'Trajeto'}

tb_tpcat = {1: 'Inicial',
            2: 'Reabertura',
            3: 'Comunicação de óbito'}

tb_tplocal_acidente = {'1': 'Estabelecimento do empregador no Brasil',
                       '2': 'Estabelecimento do empregador no exterior',
                       '3': 'Estabelecimento de terceiros onde o empregador presta serviços',
                       '4': 'Via pública',
                       '5': 'Área rural',
                       '6': 'Embarcação',
                       '9': 'Outros'}

tb_lateralidade = {0: 'Não aplicável',
                   1: 'Esquerda',
                   2: 'Direita',
                   3: 'Ambas'}

tb_ideOC = {'1': 'Conselho Regional de Medicina - CRM',
            '2': 'Conselho Regional de Odontologia - CRO',
            '3': 'Registro do Ministério da Saúde - RMS'}

tb_iniciatCAT = {1: 'Empregador',
                 2: 'Ordem judicial',
                 3: 'Determinação de órgão fiscalizador'}

tb_indretif = {1: 'Original',
               2: 'Retificação'}

tb_CDEmitenteCAT = {'1': 'Empregador',
                    '2': 'Sindicato',
                    '3': 'Médico',
                    '4': 'Segurado/Dependente',
                    '5': 'Autoridade pública'}

tb_procemi = {'1': 'Aplicativo do empregador',
              '2': 'Aplicativo governamental - Simplificado Pessoa Física',
              '3': 'Aplicativo governamental - Web Geral',
              '4': 'Aplicativo governamental - Simplificado Pessoa Jurídica',
              '22': 'Aplicativo governamental para dispositivos móveis - Empregador Doméstico'}

tb_inporte = {1: 'S'}

tb_codcateg = pd.read_csv('codcateg.csv', dtype='object').set_index('CÓDIGO').to_dict()['DESCRIÇÃO']
tb_codcateg_grupo = pd.read_csv('codcateg.csv', dtype='object').set_index('CÓDIGO').to_dict()['GRUPO']
tb_pais = pd.read_csv('esocial_paises.csv').set_index('cd_pais').to_dict()['pais']
tb_tplograd = pd.read_csv('esocial_tplogradouro.csv').set_index('CDTipoLogradouro').to_dict()['TipoLogradouro']
tb_cnae = pd.read_csv('cnae.csv', dtype='str', usecols=['CDSubclasse','DSSubclasse']).set_index('CDSubclasse').to_dict()['DSSubclasse']
tb_agente_situacao = pd.read_csv('AgenteSituacao.csv', dtype='str', usecols=['CDAgenteSituacao','DSAgenteSituacao']).set_index('CDAgenteSituacao').to_dict()['DSAgenteSituacao']

connection = pyodbc.connect('Driver={SQL Server};Server=Marfim;Database=DBCAT;')

tb_municipios = pd.read_sql_query('SELECT CDMunicipio, NOMunicipio FROM DBCAT.dbo.TBMunicipio', connection).set_index('CDMunicipio').to_dict()['NOMunicipio']
tb_natureza_lesao = pd.read_sql_query('SELECT * FROM DBCAT.dbo.TBNaturezaLesao', connection).set_index('CDNaturezaLesao').to_dict()['DSNaturezaLesao']
tb_parte_atingida = pd.read_sql_query('SELECT * FROM DBCAT.dbo.TBParteAtingida', connection).set_index('CDParteAtingida').to_dict()['DSParteAtingida']
tb_cbo = pd.read_sql_query('SELECT NRCBO, DSCBO FROM DBCAT.dbo.TBCBO', connection).set_index('NRCBO').to_dict()['DSCBO']
tb_cid = pd.read_sql_query('SELECT CDCID, DSCID FROM DBCAT.dbo.TBCID UNION ALL SELECT CDCategoria, DSCategoria FROM DBCAT.dbo.TBCID', connection).drop_duplicates().set_index('CDCID').to_dict()['DSCID']

c:\Users\joao.reis\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\joao.reis\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\joao.reis\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\joao.reis\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase st

In [4]:
dict_new_columns = {'ds_tpacid': {'col': 'tpacid', 'map': tb_tpacid},
                    'ds_tplocal_acidente': {'col': 'tplocal_acidente', 'map': tb_tplocal_acidente},
                    'ds_tplograd_local_acidente': {'col': 'tplograd_local_acidente', 'map': tb_tplograd},
                    'ds_municipio_local_acidente': {'col': 'municipio_local_acidente', 'map': tb_municipios},
                    'ds_pais_local_acidente': {'col': 'pais_local_acidente', 'map': tb_pais},
                    'ds_codagntcausador': {'col': 'codagntcausador', 'map': tb_agente_situacao},
                    'ds_codsitgeradora': {'col': 'codsitgeradora', 'map': tb_agente_situacao},
                    'ds_codparteating': {'col': 'codparteating', 'map': tb_parte_atingida},
                    'ds_lateralidade': {'col': 'lateralidade', 'map': tb_lateralidade},
                    'ds_dsclesao': {'col': 'dsclesao', 'map': tb_natureza_lesao},
                    'ds_codcid': {'col': 'codcid', 'map': tb_cid},
                    'ds_ideoc': {'col': 'ideoc', 'map': tb_ideOC},
                    'ds_codcbo': {'col': 'codcbo', 'map': tb_cbo},
                    'ds_grauinstr': {'col': 'grauinstr', 'map': tb_grauinstr},
                    'ds_racacor': {'col': 'racacor', 'map': tb_racacor},
                    'ds_codcateg': {'col': 'codcateg', 'map': tb_codcateg},
                    'ds_codcateg_grupo': {'col': 'codcateg', 'map': tb_codcateg_grupo},
                    'ds_tpinsc': {'col': 'tpinsc', 'map': tb_tpinsc},
                    'ds_localtabgeral_tpinsc': {'col': 'localtabgeral_tpinsc', 'map': tb_tpinsc},
                    'ds_cnae_localtabgeral': {'col': 'cnae_localtabgeral', 'map': tb_cnae},
                    'ds_municipio_empregador': {'col': 'municipio_empregador', 'map': tb_municipios},
                    'ds_tpinsc_estab_local_acidente': {'col': 'tpinsc_estab_local_acidente', 'map': tb_tpinsc},
                    'ds_cnae_local_acidente': {'col': 'cnae_local_acidente', 'map': tb_cnae},
                    'ds_municipio_estab_local_acidente': {'col': 'municipio_estab_local_acidente', 'map': tb_municipios},
                    'ds_CDEmitenteCAT': {'col': 'CDEmitenteCAT', 'map': tb_CDEmitenteCAT},
                    'ds_iniciatcat': {'col': 'iniciatcat', 'map': tb_iniciatCAT},
                    'ds_tpcat': {'col': 'tpcat', 'map': tb_tpcat},
                    'ds_indretif': {'col': 'indretif', 'map': tb_indretif},
                    'ds_procemi': {'col': 'procemi', 'map': tb_procemi},
                    'ds_inporte': {'col': 'inporte', 'map': tb_inporte}
                    }


In [69]:
[map_dict['col'] for new_col, map_dict in dict_new_columns.items()]

['tpacid',
 'tplocal_acidente',
 'tplograd_local_acidente',
 'municipio_local_acidente',
 'pais_local_acidente',
 'codagntcausador',
 'codsitgeradora',
 'codparteating',
 'lateralidade',
 'dsclesao',
 'codcid',
 'ideoc',
 'codcbo',
 'grauinstr',
 'racacor',
 'codcateg',
 'codcateg',
 'tpinsc',
 'localtabgeral_tpinsc',
 'cnae_localtabgeral',
 'municipio_empregador',
 'tpinsc_estab_local_acidente',
 'cnae_local_acidente',
 'municipio_estab_local_acidente',
 'CDEmitenteCAT',
 'iniciatcat',
 'tpcat',
 'indretif',
 'procemi',
 'inporte']

In [65]:
{new_col: map_dict['col'] for new_col, map_dict in dict_new_columns.items()}

{'ds_tpacid': 'tpacid',
 'ds_tplocal_acidente': 'tplocal_acidente',
 'ds_tplograd_local_acidente': 'tplograd_local_acidente',
 'ds_municipio_local_acidente': 'municipio_local_acidente',
 'ds_pais_local_acidente': 'pais_local_acidente',
 'ds_codagntcausador': 'codagntcausador',
 'ds_codsitgeradora': 'codsitgeradora',
 'ds_codparteating': 'codparteating',
 'ds_lateralidade': 'lateralidade',
 'ds_dsclesao': 'dsclesao',
 'ds_codcid': 'codcid',
 'ds_ideoc': 'ideoc',
 'ds_codcbo': 'codcbo',
 'ds_grauinstr': 'grauinstr',
 'ds_racacor': 'racacor',
 'ds_codcateg': 'codcateg',
 'ds_codcateg_grupo': 'codcateg',
 'ds_tpinsc': 'tpinsc',
 'ds_localtabgeral_tpinsc': 'localtabgeral_tpinsc',
 'ds_cnae_localtabgeral': 'cnae_localtabgeral',
 'ds_municipio_empregador': 'municipio_empregador',
 'ds_tpinsc_estab_local_acidente': 'tpinsc_estab_local_acidente',
 'ds_cnae_local_acidente': 'cnae_local_acidente',
 'ds_municipio_estab_local_acidente': 'municipio_estab_local_acidente',
 'ds_CDEmitenteCAT': 'CDEmit

In [58]:
import csv
for new_col, map_dict in dict_new_columns.items():
    table = pd.DataFrame(map_dict['map'] .items(), columns=[map_dict['col'],new_col])
    table_name = map_dict["col"]
    table.to_csv(f'{table_name}.csv', quoting=csv.QUOTE_NONNUMERIC, index=False)

In [ ]:
#TODO adicionar UORG
for new_col, map_dict in dict_new_columns.items():
    col_to_map = map_dict['col']
    dict_map = map_dict['map']
    df[new_col] = df[col_to_map].map(dict_map)

# Exportação

In [20]:
import os
import jinja2
from jinja2 import Environment, Template
import pdfkit
import pandas as pd
import numpy as np
from datetime import date
from flask import Flask, render_template

app = Flask(__name__)

def render_html(row):
    template_file = "cat_html/cat.html"
    
    with open(template_file, 'r', encoding='utf-8') as f:
        template = Template(f.read())
    
    series = row[1].fillna('N/A')
    series.idade_DTAcidente = int(series.idade_DTAcidente) if pd.notna(series.idade_DTAcidente) else 'N/A'

    output_text = template.render({col:series[col] for col in df.columns})

    html_path = f'{series.meta_nr_recibo}.html'
    
    with open(html_path, 'w') as file:
        file.write(output_text) 

In [21]:
os.add_dll_directory(r"C:/Program Files/GTK3-Runtime Win64/bin")
import weasyprint

for row in df.iterrows():
    render_html(row)
    
    recibo = row[1].meta_nr_recibo
    weasyprint.HTML(f"{recibo}.html").write_pdf(f"{recibo}.pdf")

# E-mail compilado

In [72]:
df_teste.

Index(['index', 'tpinsc', 'nrinsc', 'localtabgeral_tpinsc',
       'localtabgeral_nrinsc', 'inporte', 'cnae_localtabgeral', 'razao_social',
       'municipio_empregador', 'sguf_empregador',
       ...
       'ds_municipio_empregador', 'ds_tpinsc_estab_local_acidente',
       'ds_cnae_local_acidente', 'ds_municipio_estab_local_acidente',
       'ds_CDEmitenteCAT', 'ds_iniciatcat', 'ds_tpcat', 'ds_indretif',
       'ds_procemi', 'ds_inporte'],
      dtype='object', length=109)

In [73]:
df_teste = df[df.sguf_local_acidente == 'MG']
html_table = df_teste[['dtacid','indcatobito', 'nmtrab', 'nrinsc', 'razao_social']].to_html()

In [80]:
import os
import time
from os.path import basename
from datetime import datetime
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from jinja2 import Environment, Template
import pandas as pd

# Servidor SMTP
smtp_server = 'smtp.office365.com'
port = 587

# Usuário de autenticação
auth_user = 'joao.reis@economia.gov.br'
password = 'Pjp072022'

# Remetente
sender_email = 'cgfip.dsst@economia.gov.br'

# Assunto
assunto = "Alerta de acidente"

# Template html
template_path = 'email_html/email.html'

In [ ]:
# Arquivo de log (path)
log_file = 'teste.csv'

# Cria arquivo de log, se não existir
import os

if not os.path.exists(log_file):
   with open(log_file, mode='w') as csv_file:
       csv_file.write('cnpj,razao_social,email,data_envio,status\n')

In [91]:
destinatario = 'joao.reis@economia.gov.br'
anexos_path = ['1.1.0000000015462621164.pdf','1.1.0000000015463849777.pdf']

In [93]:
# Importa arquivo html
with open(template_path, 'r', encoding='utf-8') as f:
    html_template = Template(f.read())
    
# Logo
with open('logoSIT.png', 'rb') as img:
    logo = MIMEImage(img.read())
    
# Define the image's ID as referenced above
logo.add_header('Content-ID', '<logo>')


# if recipient["cnpj"] in notificados:
#     continue

# Create a secure SSL context
context = ssl.create_default_context()

#Mescla modelo HTML com textos
html_string = html_template.render(email=destinatario,
                                    html_table=html_table,)

# Create a multipart message
message = MIMEMultipart()
message["Subject"] = assunto
message["From"] = sender_email
message["To"] = destinatario
message.attach(MIMEText(html_string, "html")) 
message.attach(logo)

for anexo_path in anexos_path:
    # Add attachment
    if os.path.isfile(anexo_path):
        with open(anexo_path, "rb") as attachment:
            anexo = MIMEApplication(attachment.read(), Name=basename(anexo_path))

        # After the file is closed
        anexo['Content-Disposition'] = f'attachment; filename="{basename(anexo_path)}"'
    
    if os.path.isfile(anexo_path):
        message.attach(anexo)

# Try to log in to server, send email and update log file
try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    server.starttls(context=context) # Secure the connection
    server.ehlo() # Can be omitted
    server.login(auth_user, password)
    server.sendmail(sender_email, destinatario, message.as_string())
    
    # with open(log_file,'a') as csv_file:
    #     csv_file.write(f'"{recipient["cnpj"]}","{recipient["razao_social"]}","{recipient["email"]}","{str(datetime.now())}","Enviado"\n')

except Exception as erro:
    print(erro)
    
    # with open(log_file,'a') as csv_file:
    #     csv_file.write(f'"{recipient["cnpj"]}","{recipient["razao_social"]}","{recipient["email"]}","{str(datetime.now())}","Falhou"\n')

finally:
    server.quit() 
    time.sleep(10)